In [ ]:
import itertools
import random
import numpy

ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
suits = ['♣', '♦', '♥', '♠']
deck = ['{}{}'.format(i[0],i[1]) for i in list(itertools.product(ranks, suits))]

ranks_weight = {'2':2, '3':3, '4':4, '5':5, '6':6, '7':7,  '8':8,  '9':9,  'T':10,  'J':11,  'Q':12,  'K':13, 'A':14}
suits_weight = {'♣':1, '♦':2, '♥':3, '♠':4}

def get_sorted_rank(H):
    return sorted(H, key=lambda k: ranks_weight[k[0]], reverse=True)

def get_sorted_suit(H):
    return sorted(H, key=lambda k: suits_weight[k[1]], reverse=True)

def highest_rank(H):
    return H[:1], H[1:]

def highest_pair(H):
    r = [],H
    for i in range(len(H)-1):
        if H[i][0]==H[i+1][0]:
            r = H[i:i+2],H[0:i]+H[i+2:]
            break
    return r

def highest_2_pairs(H):
    r = [],H
    a,b = highest_pair(H[:])
    c,d = highest_pair(b[:]) if a else (False, None)
    if c: r = a+c,d
    return r

def highest_three(H):
    r = [],H
    for i in range(len(H)-2):
        if H[i][0]==H[i+1][0]==H[i+2][0]:
            r = H[i:i+3],H[0:i]+H[i+3:]
            break
    return r

def highest_straight(H):
    r = [],[i for i in H]
    v0 = 6 if H[0][0]=='A' else 0
    U,D = [],[]
    last = H.pop(0)
    U.append(last)
    for i in range(len(H)):
        new = H.pop(0)
        if last[0]==new[0]:
            D.append(new)
        else:
            U.append(new)
            last = new
    v1=v2=v3=v4=v5=-1
    for i in range(len(U)-4):
        v1 = ranks_weight[U[i][0]]
        v2 = ranks_weight[U[i+1][0]]+1
        v3 = ranks_weight[U[i+2][0]]+2
        v4 = ranks_weight[U[i+3][0]]+3
        v5 = ranks_weight[U[i+4][0]]+4
        if v1==v2==v3==v4==v5:
            r = U[i:i+5],get_sorted_rank(U[i+5:]+U[0:i]+D)
            break
    if v0==v2==v3==v4==v5:
        r = U[-4:]+U[0:1],get_sorted_rank(D+U[1:-4])

    return r

def highest_flush(H):
    H = get_sorted_suit(H[:])
    r = [],H
    for i in range(len(H)-4):
        if H[i][1]==H[i+1][1]==H[i+2][1]==H[i+3][1]==H[i+4][1]:
            r = get_sorted_rank(H[i:i+5]), get_sorted_rank(H[0:i]+H[i+5:])
            break
    return r

def highest_full_house(H):
    r = [],H
    a,b = highest_three(H[:])
    c,d = highest_pair(b[:]) if a else (False, None)
    if c: r = a+c,d
    return r

def highest_four(H):
    r = [],H
    for i in range(len(H)-3):
        if H[i][0]==H[i+1][0]==H[i+2][0]==H[i+3][0]:
            r = H[i:i+4],H[0:i]+H[i+4:]
            break
    return r

def _highest_count_suit(H):
    S = {}
    for k,v in suits_weight.items():
        S[v] = []
    for i in range(len(H)):
        new = H.pop(0)
        S[suits_weight[new[1]]].append(new)
    m = -1
    l = -1
    for k,v in suits_weight.items():
        x = len(S[v])
        if x>m: 
            m=x
            l=v
    r1,r2 =[],[]
    for k,v in suits_weight.items():
        if v==l:
            r1 = r1 + S[v]
        else:
            r2 = r2 + S[v]
    return get_sorted_rank(r1),get_sorted_rank(r2)

def highest_straight_flush(H):
    r = [],H
    a,b = _highest_count_suit(H[:])
    c,d = highest_straight(a[:]) if len(a)>=5 else (False, None)
    if c: r = c,get_sorted_rank(b+d)
    return r

def highest_royal_straight_flush(H):
    r = [],H
    a,b = highest_straight_flush(H[:])
    if a and a[0][0]=='A': r = a,b
    return r

hand_names = {
    1: 'highest_rank',
    2: 'highest_pair',
    3: 'highest_2_pairs',
    4: 'highest_three',
    5: 'highest_straight',
    6: 'highest_flush',
    7: 'highest_full_house',
    8: 'highest_four',
    9: 'highest_straight_flush',
    10:'highest_royal_straight_flush'
}

def check_hand(t, h=[], check=False):
    H = h+t
    hand = get_sorted_rank(H)
    ranking = {
        10:highest_royal_straight_flush,
        9: highest_straight_flush,
        8: highest_four,
        7: highest_full_house,
        6: highest_flush,
        5: highest_straight,
        4: highest_three,
        3: highest_2_pairs,
        2: highest_pair,
        1: highest_rank
    }
    
    rank, best_hand = 0, []
    
    for i in range(10,0,-1):
        h,r = ranking[i](hand[:])
        if h:
            rank, best_hand = i, h
            break
            
    tie = [ranks_weight[u[0]] for u in best_hand]
 
    return [rank]+tie, best_hand, check

def check_table(table, hand_list):
    results = []
    for i in range(len(hand_list)):
        hand = hand_list[i]
        results.append(check_hand(table, hand, i==0))
        
    results = sorted(results, key=lambda k: k[0]+[k[2]], reverse=True)
    won = results[0][2]
    tie = results[0][0]==results[1][0]
    
    return won, tie, results

def check_random(d, N=2):
    H=2
    T=5
    sample = random.sample(d, T + H*N)
    game = sample[:T],[sample[T+i*H:T+(i+1)*H] for i in range(N)]
    won, tie, result = check_table(game[0], game[1])
    result = sorted(result, key=lambda k: k[0]+[k[2]], reverse=True)
    return won, tie, result, game

def check_hand_random(d, h, t=[], N_PLAYERS=2):
    N=N_PLAYERS-1
    H=2
    T=5-len(t)
    deck_copy = d.copy()
    for card in h+t:
        deck_copy.remove(card)
    sample = random.sample(deck_copy, T+H*N)
    t = t + sample[:T]
    opponent_hands = [sample[T+i*H:T+(i+1)*H] for i in range(N)]
    won, tie, result = check_table(t, [h]+opponent_hands)
    return won, tie, result, [t, [h]+opponent_hands]

def check_hand_random_n(d, h, t=[], n=100, N_PLAYERS=2):
    N = N_PLAYERS-1
    H=2
    T=5-len(t)
    deck_copy = d.copy()
    for card in h+t:
        deck_copy.remove(card)
    dict_turns = {'won':0.0, 'tie':0.0, 'lost':0.0}
    step = 1/n
    for i in range(n):
        sample = random.sample(deck_copy, T+H*N)
        tf = t + sample[:T]
        opponent_hands = [sample[T+i*H:T+(i+1)*H] for i in range(N)]
        won, tie, result = check_table(tf, [h]+opponent_hands)
        if not won:
            dict_turns['lost'] += step
        elif won and not tie:
            dict_turns['won'] += step
        elif won and tie:
            dict_turns['tie'] += step
            
    for key in dict_turns.keys():
        dict_turns[key] = round(dict_turns[key], 3)
        
    return dict_turns

def gen_game(N=2, H=2, T=5):
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
    suits = ['♣', '♦', '♥', '♠']
    deck = ['{}{}'.format(i[0],i[1]) for i in list(itertools.product(ranks, suits))]
    sample = random.sample(deck, T + H*N)
    return sample[:T],[sample[T+i*H:T+(i+1)*H] for i in range(N)],deck

def run_turn(N=2, H=2, T=5, show=3, n=10000):
    t,hands,deck = gen_game(N, H, T)
    print(get_sorted_rank(t[:show]),hands[0])
    r = float(input('prob: '))
    s = check_table(t, hands)
    state = 'lost'
    if s[0] and not s[1]:
        state='won'
    elif s[0] and s[1]:
        state='tie'
    print('state: ', state)
    print('table: ', get_sorted_rank(t))
    for i in range(len(s[2])):
        print('P{}{}: '.format(i,'*' if s[2][i][2] else ''), s[2][i][1])

    p = check_hand_random_n(deck, hands[0],t[:show], n, N)
    print(p)
    print('expected: ', round(N*p['won']+0.5*N*p['tie']-p['lost'], 3))
    print()
    return r-p['won']-0.5*p['tie']

In [ ]:
l = []
for i in range(100):
    l.append(run_turn(2))
    x = input('finish: ')
    if x: break
round(numpy.mean(l),2),round(numpy.std(l),2)

In [ ]:
txt = """
1suit 45 %

2samesuit 6 %

1rank - 4.2% each in deck
    1specific 4.2 %

2rank - 0.37% each in deck
    2specific 0.1 %
    
gambler's ruin'

if p=0.5 => $p = n1/(n1+n2)$

"""

In [95]:
"""
class Hand -> get sorted, highest, hand names

class game -> new, random, turns, etc
"""

import random
import itertools
import numpy

class Cards:
    
    def __init__(self, deck = None):
        self.ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']

        self.suits = ['♣', '♦', '♥', '♠']

        self._start_ranks = 2
        
        self.ranks_weight = {self.ranks[i]:i+self._start_ranks for i in range(len(self.ranks))}

        self._start_suits = 1
        
        self.suits_weight = {self.suits[i]:i+self._start_suits for i in range(len(self.suits))}

        self.all = [
                '{}{}'.format(i[0], i[1])
                for i in list(itertools.product(self.ranks, self.suits))
            ]

        self.N = len(self.all)

        self.deck = deck if deck else random.sample(self.all, self.N)

    def shuffle(self, s=None, e=None):
        if not s or s<0: 
            s = 0
        if not e or e>self.N:
            e = self.N
        self.deck = self.deck[:s]+random.sample(self.deck[s:e], e-s)+self.deck[e:]


class Poker:

    cards = Cards()
    
    def __init__(self, num_players=2, my_deck=None):
        self.new_game(num_players, my_deck)

    def new_game(self, num_players=2, my_deck=None):

        self.cards = Cards(my_deck)

        if num_players * self.max_hand + self.max_table <= self.cards.N:
            self.num_players = num_players
        else:
            raise Exception("Not enough cards")

    def player(self, i):
        x = i*self.max_hand
        return self.cards.deck[x:x+self.max_hand] if i<self.num_players else []

    def table(self):
        return self.cards.deck[-self.max_table:]

    def shuffle(self, nplayers=True, ntable=True):
        
        if nplayers is True or nplayers<0 or nplayers>self.num_players:
            nplayers = self.num_players
        if ntable is True or ntable<0 or ntable>self.max_table:
            ntable = self.max_table
        self.cards.shuffle( nplayers*self.max_hand, self.cards.N - ntable )

    def set_game(self, players, table=[]):

        to_shuffle = set(self.cards.all)
        to_shuffle = to_shuffle.difference(table)
        hands = []
        for hand in players:
            hands += hand
        to_shuffle = list(to_shuffle.difference(hands))
        random.shuffle(to_shuffle)
        my_deck = hands + to_shuffle + table
        if not len(my_deck)==len(self.cards.all):
            raise Exception("Invalid game, check for repeated cards on hand/table")
        self.new_game(num_players=self.num_players, my_deck=my_deck)

    @classmethod
    def _sorted_rank(cls, H):
        return sorted(H, key=lambda k: cls.cards.ranks_weight[k[0]], reverse=True)

    @classmethod
    def _sorted_suit(cls, H):
        return sorted(H, key=lambda k: cls.cards.suits_weight[k[1]], reverse=True)
    
    @classmethod
    def sorted_hand(cls, H):
        return cls._sorted_rank(cls._sorted_suit(H))

    @classmethod
    def highest_rank(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        return (H[:1], H[1:])

    @classmethod
    def highest_pair(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        r = ([], H)
        n = len(H)
        for i in range(n-1):
            if H[i][0]==H[i+1][0]:
                r = (H[i:i+2], H[:i]+H[i+2:])
                break
        return r

    @classmethod
    def highest_2_pairs(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        a,b = cls.highest_pair(H, False)
        c,d = cls.highest_pair(b, False) if a else ([], [])
        r = (a+c, d) if c else ([], H)
        return r

    @classmethod
    def highest_three(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        r = ([], H)
        n = len(H)
        for i in range(n-2):
            if H[i][0]==H[i+1][0]==H[i+2][0]:
                r = (H[i:i+3], H[:i]+H[i+3:])
                break
        return r

    @classmethod
    def highest_straight(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        r = ([], H)
        v = [0, 0, 0, 0, 0, 0]
        v[5] = 4+cls.cards._start_ranks if H[0][0]==cls.cards.ranks[-1] else -1
        U,D = ([], [])
        last = H[0]
        U.append(last)
        for i in range(1,len(H)):
            new = H[i]
            if last[0]==new[0]:
                D.append(new)
            else:
                U.append(new)
                last = new
        for i in range(len(U)-4):
            for x in range(5):
                v[x] = cls.cards.ranks_weight[U[i+x][0]]+x
            if v[0]==v[1]==v[2]==v[3]==v[4]:
                r = (U[i:i+5], cls.sorted_hand(U[:i]+U[i+5:]+D))
                break
        if v[1]==v[2]==v[3]==v[4]==v[5]:
            r = (U[:1]+U[-4:], cls.sorted_hand(U[1:-4]+D))

        return r

    @classmethod
    def highest_flush(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        r = ([], H)
        X = cls._sorted_suit(cls._sorted_rank(H))
        n = len(X)
        for i in range(n-4):
            if X[i][1]==X[i+1][1]==X[i+2][1]==X[i+3][1]==X[i+4][1]:
                r = (X[i:i+5], cls.sorted_hand(X[:i]+X[i+5:]))
                break
        return r

    @classmethod
    def highest_full_house(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        a,b = cls.highest_three(H, False)
        c,d = cls.highest_pair(b, False) if a else ([], [])
        return (cls.sorted_hand(a+c), d) if c else ([], H)

    @classmethod
    def highest_four(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        r = ([], H)
        n = len(H)
        for i in range(n-3):
            if H[i][0]==H[i+1][0]==H[i+2][0]==H[i+3][0]:
                r = (H[i:i+4], H[:i]+H[i+4:])
                break
        return r

    @classmethod
    def highest_straight_flush(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        r = ([], H)
        for suit in cls.cards.suits[::-1]:
            U = [i for i in H if i[1]==suit]
            if len(U)>=5:
                D = [i for i in H if i[1]!=suit]
                a,b = cls.highest_straight(U, False)
                r = (a, cls.sorted_hand(D+b))
                break
        return r

    @classmethod
    def highest_royal_straight_flush(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        a,b = cls.highest_straight_flush(H, False)
        return (a,b) if a and a[1][0]==cls.cards.ranks[-2] else ([],H)
        
    @classmethod
    def compare_hands(cls, hand_a, hand_b):
        a = cls.evaluate_hand(hand_a)
        b = cls.evaluate_hand(hand_b)
        return numpy.sign(a-b)

    @classmethod
    def evaluate_hand(cls, hand):
        raise Exception("classmethod evaluate_hand not implemented")

    def winner(self):
        pass


class TexasHoldem(Poker):
    
    max_hand = 2

    max_table = 5
    
    @classmethod
    def evaluate_hand(cls, hand):
        return 1

In [96]:
x = TexasHoldem(num_players=2)
x.set_game([['8♠', '2♦'], ['Q♠', '8♥']], ['4♠', 'A♥', '7♥', '9♥'])

In [99]:
for i in range(100000):
    H = random.sample(Poker.cards.all, 7)
    H = Poker.sorted_hand(H)
    a,b = x.highest_straight_flush(H, False)
    if a:
        print(a,b,H)

['6♦', '5♦', '4♦', '3♦', '2♦'] ['8♣', '7♣'] ['8♣', '7♣', '6♦', '5♦', '4♦', '3♦', '2♦']
['A♦', '5♦', '4♦', '3♦', '2♦'] ['Q♥', '4♠'] ['A♦', 'Q♥', '5♦', '4♠', '4♦', '3♦', '2♦']
['J♠', 'T♠', '9♠', '8♠', '7♠'] ['K♦', '5♣'] ['K♦', 'J♠', 'T♠', '9♠', '8♠', '7♠', '5♣']
['6♦', '5♦', '4♦', '3♦', '2♦'] ['T♠', '2♠'] ['T♠', '6♦', '5♦', '4♦', '3♦', '2♠', '2♦']
['8♣', '7♣', '6♣', '5♣', '4♣'] ['A♠', '3♦'] ['A♠', '8♣', '7♣', '6♣', '5♣', '4♣', '3♦']
['K♣', 'Q♣', 'J♣', 'T♣', '9♣'] ['K♦', '6♥'] ['K♦', 'K♣', 'Q♣', 'J♣', 'T♣', '9♣', '6♥']
['T♣', '9♣', '8♣', '7♣', '6♣'] ['7♦', '2♠'] ['T♣', '9♣', '8♣', '7♦', '7♣', '6♣', '2♠']
['7♦', '6♦', '5♦', '4♦', '3♦'] ['A♦', '9♦'] ['A♦', '9♦', '7♦', '6♦', '5♦', '4♦', '3♦']
['K♦', 'Q♦', 'J♦', 'T♦', '9♦'] ['A♠', 'K♥'] ['A♠', 'K♥', 'K♦', 'Q♦', 'J♦', 'T♦', '9♦']
['8♣', '7♣', '6♣', '5♣', '4♣'] ['K♥', '4♠'] ['K♥', '8♣', '7♣', '6♣', '5♣', '4♠', '4♣']
['9♠', '8♠', '7♠', '6♠', '5♠'] ['Q♠', 'T♦'] ['Q♠', 'T♦', '9♠', '8♠', '7♠', '6♠', '5♠']
['T♣', '9♣', '8♣', '7♣', '6♣'] ['A♠', '5♣']

In [74]:

    #Refactor
    @classmethod
    def _highest_count_suit(cls, H):
        S = {}
        for k,v in cls.cards.suits_weight.items():
            S[v] = []
        for i in range(len(H)):
            new = H.pop(0)
            S[cls.cards.suits_weight[new[1]]].append(new)
        m = -1
        l = -1
        for k,v in cls.cards.suits_weight.items():
            x = len(S[v])
            if x>m: 
                m=x
                l=v
        r1,r2 = ([],[])
        for k,v in cls.cards.suits_weight.items():
            if v==l:
                r1 = r1 + S[v]
            else:
                r2 = r2 + S[v]
        return (cls._sorted_rank(r1), cls._sorted_rank(r2))

    #refactor
    @classmethod
    def highest_straight_flush(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        a,b = cls._highest_count_suit(H)
        c,d = cls.highest_straight(a) if len(a)>=5 else (False, None)
        return (c, cls._sorted_rank(b+d)) if c else ([],H)

    #Refactor
    @classmethod
    def highest_royal_straight_flush(cls, H, sort=True):
        if sort: 
            H = cls.sorted_hand(H)
        a,b = cls.highest_straight_flush(H)
        return (a,b) if a and a[0][0]=='A' else ([],H)
        

IndentationError: unexpected indent (<ipython-input-74-ff800e510f03>, line 3)